## Save & load Pytorch networks and Create Video using learned Policy

In [ ]:
import numpy as np
import io
import base64
from IPython.display import HTML
from gym import wrappers

Function for saving and loading pytorch network

In [ ]:
# add these to your Actor-Critic class

def save(self, directory1, directory2):
    torch.save(self.pi.state_dict(), directory1)
    torch.save(self.v.state_dict(), directory2)

def load(self, directory1, directory2):
    self.pi.load_state_dict(torch.load(directory1))
    self.v.load_state_dict(torch.load(directory2))

Example usage of the functions

In [ ]:
# save the policy you start with (you can also load the one you ended with in previous runs)
directory1 = "./LunarLanderContinuous-v2_first_actor.pth"
directory2 = "./LunarLanderContinuous-v2_first_critic.pth"
ac.save(directory1, directory2)

    #perform reinforcement learning loop

# save the policy you end with
directory1 = "./LunarLanderContinuous-v2_final_actor.pth"
directory2 = "./LunarLanderContinuous-v2_final_critic.pth"
ac.save(directory1, directory2)

Create video from previous learned policy

In [ ]:
# make test environment
test_env = gym.make("LunarLanderContinuous-v2")

# load previous policy
# use your own actor-critic network
net = MLPActorCritic(test_env.observation_space, test_env.action_space, hidden_sizes=(32,32), activation=nn.ReLU)
directory1 = "./LunarLanderContinuous-v2_first_actor.pth"
directory2 = "./LunarLanderContinuous-v2_first_critic.pth"
net.load(directory1, directory2)

# run test episodes
ep_rets = []
num_test_eps = 5
frames = []
test_env = wrappers.Monitor(test_env, "./gym-results", force=True)
for j in range(num_test_eps):
    o, d, ep_ret = test_env.reset(), False, 0
    while not d:
        a, logp = net.step(torch.as_tensor(o, dtype=torch.float32)) 
        o, r, d, _ = test_env.step(a) 
        ep_ret += r

    ep_rets.append(ep_ret)
test_env.close() # do not forget
print("Average test reward: ", np.mean(ep_rets))

# make the video
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % test_env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))